In [77]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
sys.path.append("..")
from src import sp_analisis as sa
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Ejercicio 4. Querys avanzadas 
#### Nuestro jefe acaba de venir y nos ha hecho una serie de peticiones sobre la base de datos que tenemos que poder contestar.
- #### Quiere saber cuándo fue la última vez que se pidió un producto de cada categoría.

In [78]:
sql_query =''' SELECT c.category_name,
                       MAX(o.order_date) AS "ultima_compra"
                FROM categories AS c
                INNER JOIN "products" AS p 
                ON c.category_id = p.category_id
                INNER JOIN "order_details" AS od 
                ON p.product_id = od.product_id
                INNER JOIN "orders" AS o 
                ON od.order_id = o.order_id
                GROUP BY c.category_name'''

df_categories = sa.pasos_querys(sql_query)
df_categories

,category_name,ultima_compra
0,Beverages,1998-05-06
1,Produce,1998-05-06
2,Condiments,1998-05-06
3,Grains/Cereals,1998-05-06
4,Meat/Poultry,1998-05-06
5,Confections,1998-05-06
6,Dairy Products,1998-05-06
7,Seafood,1998-05-06


----
- #### Necesita saber si existe algún producto que nunca se haya vendido por su precio original.

      Hay un producto que nunca se ha vendido por su precio original Genen Shouyu

In [79]:
query = ''' SELECT "product_id",
            "product_name",
            MAX("unit_price") AS "price_max"
           FROM "products" AS p
           GROUP BY "p"."product_id"
           HAVING MAX(unit_price) <> (
             SELECT MAX(unit_price) 
             FROM "order_details" AS "o"
             WHERE "p"."product_id" = "o"."product_id")'''
df_dif_price= sa.pasos_querys(query)
df_dif_price

,product_id,product_name,price_max
0,15,Genen Shouyu,13.0


----
- #### Quiere tener toda la información necesaria para poder identificar un tipo de producto. En concreto, tienen especial interés por los productos con categoría "Confections". Devuelve el ID del producto, el nombre del producto y su ID de categoría.

In [96]:
query = '''SELECT "product_id",
                   "product_name",
                   "category_id" 
           FROM "products" AS "p"
           WHERE "category_id" = (
             SELECT "category_id" 
             FROM "categories" 
             WHERE "category_name" = 'Confections')'''
df_confections = sa.pasos_querys(query)
df_confections

,product_id,product_name,category_id
0,16,Pavlova,3
1,19,Teatime Chocolate Biscuits,3
2,20,Sir Rodney's Marmalade,3
3,21,Sir Rodney's Scones,3
4,25,NuNuCa Nuß-Nougat-Creme,3
5,26,Gumbär Gummibärchen,3
6,27,Schoggi Schokolade,3
7,47,Zaanse koeken,3
8,48,Chocolade,3
9,49,Maxilaku,3


----
- #### Quiere saber si existe algún proveedor del que pueda prescindir ya que todos los productos que tiene se encuentran descontinuados.
            Podriamos prescindir de Refrescos Americanas LTDA porque todos sus productos están descontinuados

In [97]:
query = ''' SELECT s.company_name,
                   COUNT (p.discontinued) - sum(p.discontinued) AS "prod_no_descontinuados"
            FROM suppliers as s
            INNER JOIN products as p
            ON s.supplier_id = p.supplier_id
            GROUP BY s.company_name
            ORDER BY prod_no_descontinuados'''
df_disc = sa.pasos_querys(query)
df_disc

,company_name,prod_no_descontinuados
0,Refrescos Americanas LTDA,0
1,Nord-Ost-Fisch Handelsgesellschaft mbH,1
2,Escargots Nouveaux,1
3,Exotic Liquids,1
4,Leka Trading,2
5,Tokyo Traders,2
6,Pasta Buttini s.r.l.,2
7,PB Knäckebröd AB,2
8,Forêts d'érables,2
9,Aux joyeux ecclésiastiques,2


----
- #### Extraer los clientes que compraron mas de 30 articulos "Chai" en un único pedido

In [103]:
query = '''SELECT o."order_id", 
					  o."customer_id", 
             c."company_name" 
FROM "orders" AS o
INNER JOIN "customers" AS c
ON o."customer_id" = c."customer_id"
WHERE "order_id" IN (
	SELECT "order_id"
	FROM "order_details"
	WHERE "quantity">= 30 
	AND "product_id" = (
		SELECT "product_id"
		FROM "products"
		WHERE "product_name" = 'Chai'));'''
df_clients_chai = sa.pasos_querys(query)
df_clients_chai

,order_id,customer_id,company_name
0,10285,QUICK,QUICK-Stop
1,10522,LEHMS,Lehmanns Marktstand
2,10689,BERGS,Berglunds snabbköp
3,10691,QUICK,QUICK-Stop
4,10729,LINOD,LINO-Delicateses
5,10847,SAVEA,Save-a-lot Markets
6,10869,SEVES,Seven Seas Imports
7,10918,BOTTM,Bottom-Dollar Markets
8,11031,SAVEA,Save-a-lot Markets
9,11070,LEHMS,Lehmanns Marktstand


----
- #### Indica los clientes cuya suma total de carga en los pedidos sea mayor de 1000

In [86]:
query = ''' SELECT "customer_id",
                    "company_name"
            FROM customers 
            WHERE "customer_id" IN (
              SELECT "customer_id" 
              FROM "orders" 
              GROUP BY "customer_id" 
              HAVING SUM("freight") > 1000);'''
df_clients = sa.pasos_querys(query)
df_clients

,customer_id,company_name
0,BERGS,Berglunds snabbköp
1,BONAP,Bon app'
2,ERNSH,Ernst Handel
3,FOLKO,Folk och fä HB
4,FRANK,Frankenversand
5,GREAL,Great Lakes Food Market
6,HILAA,HILARION-Abastos
7,HUNGO,Hungry Owl All-Night Grocers
8,LEHMS,Lehmanns Marktstand
9,MEREP,Mère Paillarde


----
- #### Desde recursos humanos nos piden seleccionar los nombres de las ciudades con 5 o más empleadas de cara a estudiar la apertura de nuevas oficinas

      No hay ciudades con 5 o más empleadas, solo está London que tiene 4 empleadas.

In [88]:
query = ''' SELECT "city",
                    COUNT("employee_id") AS "num_employees"
            FROM "employees"
            GROUP BY "city"
            HAVING COUNT("city") >= 4
'''
df_store = sa.pasos_querys(query)
df_store

,city,num_employees
0,London,4
